##### Trajectory Computer
A purely computational model that has no I/O, database calls, or FastAPI dependencies. It receives pre-loaded data points and user-input zome parameters, and runs the 2-derivative analysis framework, returning a fully computed result dict.  

f(x)   — health score h(raw) classified into a zone (Non-pathology / Vulnerability / Pathology)
f'(x)  — first derivative: direction of change (Improving / Stable / Worsening)
f''(x) — second derivative: curvature (Accelerating / Steady / Decelerating)

Raw marker values are first normalized to a scalar health score h via a
u-shaped transform:

    mid        = (healthy_min + healthy_max) / 2
    half_range = (healthy_max - healthy_min) / 2
    h(raw)     = 1 - |raw - mid| / half_range

    h = 1.0  when raw = mid (optimal centre of healthy range)
    h = 0.0  when raw = healthy_min or healthy_max (at the healthy edge)
    h < 0    when raw is outside the healthy range

Because h > 0 is always healthier (regardless of whether the underlying marker
should be high or low), f'(x) > 0 always means "improving" without any
per-marker direction logic. The polynomial is fitted to h values; all
derivatives and zone assignments operate in h-space.

These three sign-classes combine into one of 27 discrete trajectory states.

In [1]:
import numpy as np
from datetime import datetime 

**Constants:** 

Derivatives whose absolute value is below the defined threshold are classified as 0 (To avoid ever so slight trends being detected as significant momentum).

numpy.roots always returns complex numbers, this constant defines a value for the imaginary component below which the root is considered a real number

In [ ]:
DERIVATIVE_ZERO_THRESHOLD = 0.001 #units: health-units/hour
IMAGINARY_TOLERANCE = 1e-6

**Normalization:**

This function maps a measured marker value to a scalar "h-score" via a u-shaped transform. The formula is:

    h = 1 - |raw - mid| / half_range

Properties: 

h = 1.0 when raw = mid (exactly at healthy optimum)
h = 0.0 when raw = healthy_min or healthy_max (at the healthy boundary)
h < 0 when raw is outside of the healthy range
h = -1.0 when raw deviates by a further half_range beyond the boundary.

Since higher h always means healthier (regardless of whenther the marker "should" be high or low), all subsequent logic (polynomial fitting, derivative interpretation, zone assignment) is direction-agnostic.

(mid == center of healthy range, half_range == distance from mid to boundary)

In [3]:
def _normalize(raw: float, mid: float, half_range: float) -> float:
    return 1.0 - abs(raw - mid) / half_range

**Assign zone from h-score:**

Classifies a health score into one of three named zones:

Zone boundaries: 
    h > vulnerability_margin   -> non_pathology
    |h| <= vulnerability_margin   -> vulnerability
    h < vulnerability_margin   -> pathology 

Note: zone assignment always uses h computed from the raw measured value, not from the fitted polynomical value.The polynomial smooths noise for calculating derivatives so in some cases may shift the placement into a new zone, zone assignment should reflect the actual measurement.

In [4]:
def _assign_zone_from_score(h: float, vulnerability_margin: float) -> str:
    if h > vulnerability_margin:
        return "non_pathology"
    if h < -vulnerability_margin:
        return "pathology"
    return "vulnerability"

**Derivative sign classification:**

Maps a derivative value to a ternary sign class. Returns:
    +1 if value > DERIVATIVE_ZERO_THRESHOLD ("imporving"/"accelerating")
    0 if |value| <= DERIVATIVE_ZERO_THRESHOLD ("Stable"/"Steady")
    -1 if value < DERIVATIVE_ZERO_THRESHOLD ("Worsening"/"Decelerating")

The threshold prevents noise in almost flat reagions causing the program to interpret it as meaningful directional movement. The units are h-units/hour, so the threshold applies to the normalized values and doesn't consider the raw scale.

In [2]:
def _sign_class(value: float) -> int:
    if value > DERIVATIVE_ZERO_THRESHOLD:
        return 1
    if value < -DERIVATIVE_ZERO_THRESHOLD:
        return -1
    return 0

**Trajectory state number:**

Three separate lookup dictionaries encode the 27-state indexing scheme. They take the sign returned from *_sign_class()* (-1, 0, or 1) and assigns an "offest" value to be added that results in the final state number. 

**f offset:** if sign 1, add 0; if sign 0, add 9; if sign -1, add 18

**f_prime offset:** if sign 1, add 0; if sign 0, add 3; if sign -1, add 6

**f_double_prime offset:** if sign 1, add 1; if sign 0, add 2; if sign -1, add 3

These get added together to determine with of the 27 states the point is in.

Ex: (f==-1, fp==-1, fpp==1) -> 18 + 6 + 1 == 25. State #25 is *Decelerating Worsening Pathology* 

In [6]:
_ZONE_OFFSET = {1: 0, 0: 9, -1: 18}
_FP_OFFSET = {1: 0, 0: 3, -1: 6}
_FPP_INDEX = {1: 1, 0: 2, -1: 3}

def _trajectory_state(zone: str, fp_sign: int, fpp_sign: int) -> int:
    zone_sign = {"non_pathology": 1, "vulnerability": 0, "pathology": -1}[zone]
    return _ZONE_OFFSET[zone_sign] + _FP_OFFSET[fp_sign] + _FPP_INDEX[fpp_sign]

**Time-to-zone Transition:**

Estimates number of hours from a given point in time that the fitted polynomial will next cross any zone boundary. There are 3 boundary values in h-space:

    +vulnerability_margin -> upper edge of vulnerability/non=pathology boundary
    0                     -> h=o line (raw value at heathy_min or healthy _max)
    -vulnerability_margin -> lower edge of vulnerability/pathology boundary

Algorithm _time_to_transition():

For each boundary value B:

 1. Copies the polynomial coefficients and subtracts B from the constant term, so that the problem lies in identifying the fitted curves x-intercepts instead of where f(x)=B. 
 2. Calls numpy.roots on the shifted polynomial to find all roots (returns complex number)
 3. Ignores any complex roots using IMAGINARY_TOLERANCE threshold constant.
 4. Ignores any past and present roots (we only care about the future)
 5. Returns the smallest of the remaining roots (reflects the nearest future boundary crossing time)

The parameters are as follows:
- coeffs: lists of polynomical coefficientsm in order of decreasing degree
- vulnerability_margin: h-space scalar (e.g. 0.1)
- x_current: x-coordinate of data point being currently analyzed

In [7]:
def _time_to_transition(
    coeffs: np.ndarray,
    vulnerability_margin: float,
    x_current: float,
) -> float | None:
    
    candidate_times = []

    for boundary_value in (vulnerability_margin, 0.0, -vulnerability_margin):
        shifted = coeffs.copy()
        shifted[-1] -= boundary_value

        roots = np.roots(shifted)

        for root in roots:
            if abs(root.imag) > IMAGINARY_TOLERANCE:
                continue

            t = root.real

            if t > x_current:
                candidate_times.append(t)

    return float(min(candidate_times)) if candidate_times else None

**Compute trajectory:**

Runs all of the functions described above in a six-step sequence:

1. **Extract parameters and compute normalization constants:**
healthy_min, healthy_max, and vulnerability_margin are pulled from zone_boundaries. mid and half_range are computed once and reused for all future calculations.

2. **Build time and h-score arrays:**
The first datapoint's timestamp becomes t0, with every subsequent timestamp is expressed as hours elapsed since t0. Also, every raw value is normalized to h-score using _normalize(). Both of these results are returned as numpy arrays.

3. **Fit the polynomial:**
numpy.polyfit() fits a least-squared polynomial of the requested degree to the datapoints. Returns an array of coefficient values.

4. **Devrive first and second derivative polynomials:**
numpy.polyder(coeffs, 1) returns the first derivative polynomial; numpy.polyder(coeffs, 2) returns the second derivative polynomial.

5. **Compute per-datapoint results:**
For each datapoint sequentially, the function:
    1. Retrieves the precomputed h-score
    2. Evaluates the polynomial at x to get the fitted value at that point
    3. Evaluates the 1st and 2nd derivative polynomials at x to get the values at those points
    4. Assigns the zone from raw health value
    5. Classifies both derivatives into sign classes
    6. Looks up the trajectory state (1-27)
    7. Calls _time_to_transition() to find nearest future boundary crossing for fitted polynomial
    8. Packages all of that into a dict and appends result_points array using the specified format

6. **Assemble fit_metadata:**
A seconds output dict is created in a way that the Svelte frontend can best use.  



**compute_trajectory() arguments and what it returns:**

It takes as parameters the data_points dict returned from the *archive_reader.read_timeseries()* function. The zone_boundaries dict is manually input by the user on the frontend, as is polynomial degree

It returns a dict with two keys:

            "data_points" : list[dict]
            One entry per input data point, each with:
                "timestamp"                 — ISO 8601 string
                "x_hours"                   — float, hours since earliest data point
                "raw_value"                 — float, original measured value
                "health_score"              — float, h(raw_value); 1.0=optimal, 0.0=at edge
                "fitted_value"              — float, h-polynomial evaluated at x_hours
                "zone"                      — "non_pathology" | "vulnerability" | "pathology"
                "f_prime"                   — float, first derivative at x_hours (h-units/hour)
                "f_double_prime"            — float, second derivative at x_hours
                "trajectory_state"          — int 1–27
                "time_to_transition_hours"  — float or None

        "fit_metadata" : dict
            Data the frontend needs to evaluate the curve client-side:
                "coefficients"      — list[float], numpy.polyfit output (h-units, highest degree first)
                "t0_iso"            — ISO 8601 string, the timestamp corresponding to x=0
                "polynomial_degree" — int
                "normalization"     — dict with mid, half_range, healthy_min, healthy_max
                                      for client-side raw→h conversion
                "zone_boundaries"   — dict with vulnerability_margin (h-space scalar)